# VGG块

VGG块的组成规律是：连续使用数个相同的填充为1、窗口形状为3*3的卷积层后接一个歩幅为2的最大池化层。卷积层保持输入的h和w不变，而池化层使h和w减半。

In [1]:
import torch
from torch import nn 

def vggBlock(numConvs,inChannels,outChannels):
    vggBlock=[]
    for i in range(numConvs):
        vggBlock.append(nn.Conv2d(inChannels if i==0 else outChannels,outChannels,3,1,1))
        vggBlock.append(nn.ReLU())
    vggBlock.append(nn.MaxPool2d(2,2))
    return nn.Sequential(*vggBlock)

# VGG网络

卷积层模块串联数个vggBlock，其超参数由变量conv_arch定义。该变量指定了每个VGG块里的卷积层个数和输入输出通道数。全连接模块和AlexNet中的一样

In [2]:
conv_arch = ((1,1,64),(1,64,128),(2,128,256),(2,256,512),(2,512,512))
fc_features=512*7*7
fc_hidden_units=4096

VGG_11

In [3]:
def vgg(conv_arch,fc_features,fc_hidden_units=4096):
    net = nn.Sequential()
    for i ,(numConvs,inChannels,outChannels) in enumerate(conv_arch):
        net.add_module(str(i),vggBlock(numConvs,inChannels,outChannels))
    net.add_module('fc',nn.Sequential(
        nn.Flatten(),
        nn.Linear(fc_features,fc_hidden_units),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(fc_hidden_units,fc_hidden_units),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(fc_hidden_units,10)
    ))
    return net

In [4]:
net = vgg(conv_arch,fc_features,fc_hidden_units)
x=torch.rand(1,1,224,224)

for name,layer in net.named_children():
    x=layer(x)
    print(name,x.shape)

0 torch.Size([1, 64, 112, 112])
1 torch.Size([1, 128, 56, 56])
2 torch.Size([1, 256, 28, 28])
3 torch.Size([1, 512, 14, 14])
4 torch.Size([1, 512, 7, 7])
fc torch.Size([1, 10])


因为VGG-11比AlexNet计算上更加复杂，出于测试目的，我们构造一个通道数更小，或者说更窄的网络在FashionMNIST数据集上进行测试。

In [6]:
ratio=8
small_conv_arch = ((1,1,64//ratio),(1,64//ratio,128//ratio),(2,128//ratio,256//ratio),(2,256//ratio,512//ratio),(2,512//ratio,512//ratio))
net=vgg(small_conv_arch,fc_features//8,fc_hidden_units)
print(net)

Sequential(
  (0): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (4): Sequential